In [1]:
from datasets import Dataset, DatasetDict
from pyvi import ViTokenizer
import pandas as pd
import numpy as np
import random
import re

In [2]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

In [3]:
load_dotenv()
login(token=os.getenv("UIT_21520296_DATASET"))

# helper functions

In [4]:
def preprocess_with_segmentation (element):
    # Replace newlines, tabs, and redundant spaces with a single space
    processed_element = re.sub(r'[\n\t\s]+', ' ', element)
    
    # Strip leading and trailing spaces
    processed_element = processed_element.strip()
    
    # Tokenize the text using ViTokenizer
    segmented_element = ViTokenizer.tokenize(processed_element)
    
    return segmented_element

In [5]:
def preprocess_no_segmentation(element):
    # Replace newlines, tabs, and redundant spaces with a single space
    processed_element = re.sub(r'[\n\t\s]+', ' ', element)
    
    # Strip leading and trailing spaces
    processed_element = processed_element.strip()
    
    return processed_element

In [6]:
def make_pair_class_dataset(df):
    pair_class_data = []

    # Convert all contexts to a set for faster lookup and uniqueness
    all_contexts = set(df["context"].tolist())

    for _, row in df.iterrows():
        pair_class_data.append({"question": row["question"], "context": row["context"], "label": 1})  # Positive sample

        # Get a negative sample (context that is different from the current one)
        possible_negatives = [ctx for ctx in all_contexts if ctx != row["context"]]
        
        if possible_negatives:  # Ensure we have valid negative samples
            negative_context = random.choice(possible_negatives)
            pair_class_data.append({"question": row["question"], "context": negative_context, "label": 0})

    return Dataset.from_pandas(pd.DataFrame(pair_class_data))


In [7]:
def make_triplet_dataset(df):
    triplet_data = []
    all_contexts = set(df["context"].tolist())  # Convert to set for faster lookup
    
    for _, row in df.iterrows():
        # Get a negative sample (context that is different from the current one)
        possible_negatives = [ctx for ctx in all_contexts if ctx != row["context"]]
        
        if possible_negatives:  # Ensure negative samples are available
            negative_context = random.choice(possible_negatives)
            triplet_data.append({
                "anchor": row["question"], 
                "positive": row["context"], 
                "negative": negative_context
            })
    
    return Dataset.from_pandas(pd.DataFrame(triplet_data))

# create datasets


In [8]:
columns_to_drop = ['index', 'article', 'document', 'extractive answer', 'abstractive answer', 'yes/no']

## word-segmented format 

**1. training set**

In [91]:
df_train = pd.read_csv("[R2GQA]-data-full/train.csv")
df_train.head()

,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,8276,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG,Sinh viên dự bị không trở thành sinh viên chín...,02 học kỳ liên tiếp,Sinh viên dự bị không trở thành sinh viên chín...,NaN
1,8543,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,theo đề xuất hợp lý của bộ phận chuyên môn,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,NaN
2,7152,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...,N3,Sinh viên CT CLC phải đạt tiếng Nhật N3 thì mớ...,NaN
3,2218,Điều 25. Cách tính điểm trung bình\n1. ĐTBH...,Điều 25. Cách tính điểm trung bình,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,"Cách tính điểm giữa điểm trung bình học kỳ, đi...","ĐTBHK, ĐTBC và ĐTBCTL được tính theo công thức","Cách tính ĐTBHK, ĐTBC và ĐTBCTL được tính theo...",1.0
4,5361,Điều 19. Tổ chức phúc khảo và giải quyết khiế...,Điều 19. Tổ chức phúc khảo và giải quyết khiếu...,QUY ĐỊNH Tổ chức thi các môn học hệ đại học ch...,Hoạt động nào sẽ diễn ra khi SV có khiếu nại v...,phúc khảo,Trường sẽ tổ chức phúc khảo và giải quyết khiế...,NaN


In [92]:
df_train.drop(columns=columns_to_drop, inplace=True)
print(df_train.shape)
df_train.head()

(7806, 2)


,context,question
0,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Sinh viên dự bị không trở thành sinh viên chín...
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...
2,Điều 5. Chương trình đào tạo\nCT CLC đ...,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...
3,Điều 25. Cách tính điểm trung bình\n1. ĐTBH...,"Cách tính điểm giữa điểm trung bình học kỳ, đi..."
4,Điều 19. Tổ chức phúc khảo và giải quyết khiế...,Hoạt động nào sẽ diễn ra khi SV có khiếu nại v...


In [93]:
df_train = df_train.map(func=preprocess_with_segmentation)
df_train.head()

,context,question
0,Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_...,Sinh_viên dự_bị không trở_thành sinh_viên chín...
1,Điều 4 . Kiểm_tra xếp lớp đầu khóa cho sinh_vi...,Các mức điểm xếp lớp tiếng Anh được Hiệu_trưởn...
2,Điều 5 . Chương_trình đào_tạo CT CLC được xây_...,Trình_độ tiếng Nhật đạt N mấy mới thì sinh_viê...
3,Điều 25 . Cách tính điểm trung_bình 1 . ĐTBHK ...,"Cách tính điểm giữa điểm trung_bình học_kỳ , đ..."
4,Điều 19 . Tổ_chức phúc_khảo và giải_quyết khiế...,Hoạt_động nào sẽ diễn ra khi SV có khiếu_nại v...


**2. validation set**

In [94]:
df_valid = pd.read_csv("[R2GQA]-data-full/val.csv")
df_valid.drop(columns=columns_to_drop, inplace=True)
df_valid = df_valid.map(func=preprocess_with_segmentation)

print(df_valid.shape)
df_valid.head()

(976, 2)


,context,question
0,Điều 16 . Xử_lý học vụ Xử_lý học vụ nhằm giúp ...,Sinh_viên chưa hết thời_gian tối_đa hoàn_thành...
1,"Điều 32 . Chấm thực_tập , khóa luận tốt_nghiệp...",Điểm của khoá_luận tính theo thang điểm mấy ?
2,Điều 11 . Đào_tạo ngoại_ngữ - Vào đầu khóa học...,Vào đầu mỗi khóa học_sinh_viên CT CLC cần phải...
3,"Điều 9 . Văn_bằng , chứng_chỉ / chứng_nhận đượ...",Tại_sao sinh_viên cần nộp chứng_chỉ cho Trường ?
4,Điều 1 . Phạm_vi điều_chỉnh và đối_tượng áp_dụ...,"Văn_bằng giáo_dục đại_học , sau đại_học và các..."


**3. testing set**

In [95]:
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")
df_test.drop(columns=columns_to_drop, inplace=True)
df_test = df_test.map(func=preprocess_with_segmentation)

print(df_test.shape)
df_test.head()

(976, 2)


,context,question
0,"Điều 8 . Xây_dựng , thẩm_định học liệu điện_tử...",Học_liệu điện_tử sau khi được thông_qua bởi ĐV...
1,"Điều 20 . Hồ_sơ , trình_tự , thủ_tục chỉnh_sửa...",Có_thể nộp bản_sao giấy khai_sinh cho hồ_sơ đề...
2,Điều 23 . Điểm Miễn 1 . Điểm BL - Sinh_viên đã...,Quy_định về điểm M trong trường_hợp sinh_viên ...
3,Điều 4 . Xét_duyệt đề_tài KLTN Khoa chịu trách...,Yêu_cầu đối_với học_vị của CBHD hướng_dẫn KLTN...
4,Điều 11 . Quy_trình chuẩn_bị cho kỳ thi 1 . Đầ...,Công_việc phân_công_cán_bộ coi thi có nội_dung...


In [97]:
pair_class_train = make_pair_class_dataset(df_train)
pair_class_valid = make_pair_class_dataset(df_valid)
pair_class_test  = make_pair_class_dataset(df_test)

pair_class_dataset = DatasetDict({"train": pair_class_train, 
                                  "valid": pair_class_valid, 
                                  "test" : pair_class_test
                                })

print(pair_class_dataset)
pair_class_dataset["train"][0], pair_class_dataset["train"][1]

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 15612
    })
    valid: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 1952
    })
    test: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 1952
    })
})


({'question': 'Sinh_viên dự_bị không trở_thành sinh_viên chính_thức bao_nhiêu học_kỳ sẽ bị loại khỏi CTTN ?',
  'context': 'Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_trình , xét chính_thức và dự_bị Đối_tượng tham_gia CTTN là những sinh_viên có năng_lực xuất_sắc , do đó , sau mỗi học_kỳ BĐH quyết_định việc loại sinh_viên khỏi lớp tài_năng , tuyển bổ_sung sinh_viên từ_chương_trình chuẩn vào lớp tài_năng , xét chuyển_đổi sinh_viên chính_thức và dự_bị . Đầu mỗi học_kỳ , Khoa xét và đề_nghị lên BĐH các danh_sách sinh_viên tuyển bổ_sung , bị loại ra khỏi các lớp CTTN hoặc danh_sách sinh_viên chính_thức và dự_bị theo các tiêu_chuẩn như sau : 1 . Loại khỏi chương_trình Tại thời_điểm xem_xét , sinh_viên rơi vào một trong các trường_hợp sau : - Chưa tốt_nghiệp khi đã quá thời_gian thiết_kế của khóa học và không có_lý_do đặc_biệt . - Không đăng_ký học đầy_đủ các môn_học CTTN bắt_buộc trong học_kỳ . - ĐTBTL nhỏ hơn 6,5 – tính tương_ứng sau học_kỳ 1 và sau học_kỳ hè ( kết_quả học_tập của học_kỳ

In [64]:
pair_class_dataset.push_to_hub(repo_id='KhoaUIT/UIT-R2GQA', 
                               config_name='pair-class-segmented')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA/commit/e6dad4b16a4d4525f8b3b51d4b7c1ab0a0078353', commit_message='Upload dataset', commit_description='', oid='e6dad4b16a4d4525f8b3b51d4b7c1ab0a0078353', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KhoaUIT/UIT-R2GQA'), pr_revision=None, pr_num=None)

In [99]:
triplet_train = make_triplet_dataset(df_train)
triplet_valid = make_triplet_dataset(df_valid)
triplet_test  = make_triplet_dataset(df_test)

triplet_dataset = DatasetDict({"train": triplet_train, 
                                  "valid": triplet_valid, 
                                  "test" : triplet_test
                                })

print(triplet_dataset)
triplet_dataset["train"][0], triplet_dataset["train"][1]

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 7806
    })
    valid: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 976
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 976
    })
})


({'anchor': 'Sinh_viên dự_bị không trở_thành sinh_viên chính_thức bao_nhiêu học_kỳ sẽ bị loại khỏi CTTN ?',
  'positive': 'Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_trình , xét chính_thức và dự_bị Đối_tượng tham_gia CTTN là những sinh_viên có năng_lực xuất_sắc , do đó , sau mỗi học_kỳ BĐH quyết_định việc loại sinh_viên khỏi lớp tài_năng , tuyển bổ_sung sinh_viên từ_chương_trình chuẩn vào lớp tài_năng , xét chuyển_đổi sinh_viên chính_thức và dự_bị . Đầu mỗi học_kỳ , Khoa xét và đề_nghị lên BĐH các danh_sách sinh_viên tuyển bổ_sung , bị loại ra khỏi các lớp CTTN hoặc danh_sách sinh_viên chính_thức và dự_bị theo các tiêu_chuẩn như sau : 1 . Loại khỏi chương_trình Tại thời_điểm xem_xét , sinh_viên rơi vào một trong các trường_hợp sau : - Chưa tốt_nghiệp khi đã quá thời_gian thiết_kế của khóa học và không có_lý_do đặc_biệt . - Không đăng_ký học đầy_đủ các môn_học CTTN bắt_buộc trong học_kỳ . - ĐTBTL nhỏ hơn 6,5 – tính tương_ứng sau học_kỳ 1 và sau học_kỳ hè ( kết_quả học_tập của học_kỳ 

In [67]:
triplet_dataset.push_to_hub(repo_id='KhoaUIT/UIT-R2GQA', 
                            config_name='triplet-segmented')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/648 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA/commit/07a9f52fb01dae33e9f4c19e8d27018a73181681', commit_message='Upload dataset', commit_description='', oid='07a9f52fb01dae33e9f4c19e8d27018a73181681', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KhoaUIT/UIT-R2GQA'), pr_revision=None, pr_num=None)

## no word-segmented format

In [14]:
df_train = pd.read_csv("[R2GQA]-data-full/train.csv")
df_train.drop(columns=columns_to_drop, inplace=True)
df_train = df_train.map(func=preprocess_no_segmentation)

print(df_train.shape)
df_train.head()

(7806, 2)


,context,question
0,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,Sinh viên dự bị không trở thành sinh viên chín...
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...
2,Điều 5. Chương trình đào tạo CT CLC được xây d...,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...
3,"Điều 25. Cách tính điểm trung bình 1. ĐTBHK, Đ...","Cách tính điểm giữa điểm trung bình học kỳ, đi..."
4,Điều 19. Tổ chức phúc khảo và giải quyết khiếu...,Hoạt động nào sẽ diễn ra khi SV có khiếu nại v...


In [15]:
df_valid = pd.read_csv("[R2GQA]-data-full/val.csv")
df_valid.drop(columns=columns_to_drop, inplace=True)
df_valid = df_valid.map(func=preprocess_no_segmentation)

print(df_valid.shape)
df_valid.head()

(976, 2)


,context,question
0,Điều 16. Xử lý học vụ Xử lý học vụ nhằm giúp c...,Sinh viên chưa hết thời gian tối đa hoàn thành...
1,"Điều 32. Chấm thực tập, khóa luận tốt nghiệp 1...",Điểm của khoá luận tính theo thang điểm mấy?
2,"Điều 11. Đào tạo ngoại ngữ - Vào đầu khóa học,...",Vào đầu mỗi khóa học sinh viên CT CLC cần phải...
3,"Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử...",Tại sao sinh viên cần nộp chứng chỉ cho Trường?
4,Điều 1. Phạm vi điều chỉnh và đối tượng áp dụn...,"Văn bằng giáo dục đại học, sau đại học và các ..."


In [16]:
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")
df_test.drop(columns=columns_to_drop, inplace=True)
df_test = df_test.map(func=preprocess_no_segmentation)

print(df_test.shape)
df_test.head()

(976, 2)


,context,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử 1...",Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn 1. Điểm BL - Sinh viên đã t...,Quy định về điểm M trong trường hợp sinh viên ...
3,Điều 4. Xét duyệt đề tài KLTN Khoa chịu trách ...,Yêu cầu đối với học vị của CBHD hướng dẫn KLTN...
4,Điều 11. Quy trình chuẩn bị cho kỳ thi 1. Đầu ...,Công việc phân công cán bộ coi thi có nội dung...


In [17]:
pair_class_train = make_pair_class_dataset(df_train)
pair_class_valid = make_pair_class_dataset(df_valid)
pair_class_test  = make_pair_class_dataset(df_test)

pair_class_dataset = DatasetDict({"train": pair_class_train, 
                                  "valid": pair_class_valid, 
                                  "test" : pair_class_test
                                })

print(pair_class_dataset)
pair_class_dataset["train"][0], pair_class_dataset["train"][1]

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 15612
    })
    valid: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 1952
    })
    test: Dataset({
        features: ['question', 'context', 'label'],
        num_rows: 1952
    })
})


({'question': 'Sinh viên dự bị không trở thành sinh viên chính thức bao nhiêu học kỳ sẽ bị loại khỏi CTTN?',
  'context': 'Điều 9. Tuyển bổ sung và loại ra khỏi chương trình, xét chính thức và dự bị Đối tượng tham gia CTTN là những sinh viên có năng lực xuất sắc, do đó, sau mỗi học kỳ BĐH quyết định việc loại sinh viên khỏi lớp tài năng, tuyển bổ sung sinh viên từ chương trình chuẩn vào lớp tài năng, xét chuyển đổi sinh viên chính thức và dự bị. Đầu mỗi học kỳ, Khoa xét và đề nghị lên BĐH các danh sách sinh viên tuyển bổ sung, bị loại ra khỏi các lớp CTTN hoặc danh sách sinh viên chính thức và dự bị theo các tiêu chuẩn như sau: 1. Loại khỏi chương trình Tại thời điểm xem xét, sinh viên rơi vào một trong các trường hợp sau: - Chưa tốt nghiệp khi đã quá thời gian thiết kế của khóa học và không có lý do đặc biệt. - Không đăng ký học đầy đủ các môn học CTTN bắt buộc trong học kỳ. - ĐTBTL nhỏ hơn 6,5 – tính tương ứng sau học kỳ 1 và sau học kỳ hè (kết quả học tập của học kỳ hè sẽ được tính 

In [18]:
pair_class_dataset.push_to_hub(repo_id='KhoaUIT/UIT-R2GQA', 
                               config_name='pair-class')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\datasets--KhoaUIT--UIT-R2GQA. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA/commit/240108e3b110c4d7a34b45a0c99af742e3bf4a83', commit_message='Upload dataset', commit_description='', oid='240108e3b110c4d7a34b45a0c99af742e3bf4a83', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KhoaUIT/UIT-R2GQA'), pr_revision=None, pr_num=None)

In [19]:
triplet_train = make_triplet_dataset(df_train)
triplet_valid = make_triplet_dataset(df_valid)
triplet_test  = make_triplet_dataset(df_test)

triplet_dataset = DatasetDict({"train": triplet_train, 
                                  "valid": triplet_valid, 
                                  "test" : triplet_test
                                })

print(triplet_dataset)
triplet_dataset["train"][0], triplet_dataset["train"][1]

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 7806
    })
    valid: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 976
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 976
    })
})


({'anchor': 'Sinh viên dự bị không trở thành sinh viên chính thức bao nhiêu học kỳ sẽ bị loại khỏi CTTN?',
  'positive': 'Điều 9. Tuyển bổ sung và loại ra khỏi chương trình, xét chính thức và dự bị Đối tượng tham gia CTTN là những sinh viên có năng lực xuất sắc, do đó, sau mỗi học kỳ BĐH quyết định việc loại sinh viên khỏi lớp tài năng, tuyển bổ sung sinh viên từ chương trình chuẩn vào lớp tài năng, xét chuyển đổi sinh viên chính thức và dự bị. Đầu mỗi học kỳ, Khoa xét và đề nghị lên BĐH các danh sách sinh viên tuyển bổ sung, bị loại ra khỏi các lớp CTTN hoặc danh sách sinh viên chính thức và dự bị theo các tiêu chuẩn như sau: 1. Loại khỏi chương trình Tại thời điểm xem xét, sinh viên rơi vào một trong các trường hợp sau: - Chưa tốt nghiệp khi đã quá thời gian thiết kế của khóa học và không có lý do đặc biệt. - Không đăng ký học đầy đủ các môn học CTTN bắt buộc trong học kỳ. - ĐTBTL nhỏ hơn 6,5 – tính tương ứng sau học kỳ 1 và sau học kỳ hè (kết quả học tập của học kỳ hè sẽ được tính c

In [20]:
triplet_dataset.push_to_hub(repo_id='KhoaUIT/UIT-R2GQA', 
                            config_name='triplet')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA/commit/bfdd3ff14113299963b9d1331aeab326a80c70a7', commit_message='Upload dataset', commit_description='', oid='bfdd3ff14113299963b9d1331aeab326a80c70a7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KhoaUIT/UIT-R2GQA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KhoaUIT/UIT-R2GQA'), pr_revision=None, pr_num=None)